### Snowplow - Advanced Analytics for Web Accelerator

In this notebook we will be modeling and exploring behavioural data collected by Snowplow's Javascript tracker in Databricks. You can use the above widgets to set the start and end date of the data you would like to explore.

In [0]:
from datetime import date, timedelta

widget_start_date = str(date.today() - timedelta(days=7))
dbutils.widgets.text("start_date", widget_start_date)


In [0]:
displayHTML("""<font size="8" color="darkblue" face="sans-serif"><center>Snowplow Web Dashboard</center></font>""")

Snowplow Web Dashboard

### Overview

This notebook provides the base for your custom dashboard using the derived tables - **snowplow_web_page_views** / **snowplow_web_sessions** / **snowplow_web_users** produced by the [snowplow_web dbt package](https://docs.snowplow.io/dbt-snowplow-web/#!/overview/snowplow_web).

# PAGEVIEWS

In [0]:
timeframe = getArgument('start_date')
df = spark.sql(
    """ SELECT date(START_TSTAMP) as Date, count(*) AS number_of_pageviews
        FROM  derived.snowplow_web_page_views
        -- filter data range, if needed, example: WHERE START_TSTAMP BETWEEN DATEADD(day, -7, GETDATE()) AND  DATEADD(day, -1, GETDATE())
        GROUP BY 1
        ORDER BY 1
    """)

df.registerTempTable("pageviews_by_day")

display(spark.sql("SELECT * FROM pageviews_by_day"))



Date,number_of_pageviews
2022-08-19,41
2022-08-20,884
2022-08-21,938
2022-08-22,51


In [0]:
dbutils.widgets.removeAll()

In [0]:
df = spark.sql(
    """ SELECT PAGE_TITLE, SUM(PAGE_VIEWS_IN_SESSION) AS number_of_pageviews
        FROM derived.snowplow_web_page_views
        WHERE PAGE_TITLE IS NOT NULL
        -- filter data range, if needed, example: AND START_TSTAMP BETWEEN DATEADD(day, -7, GETDATE()) AND  DATEADD(day, -1, GETDATE())
        GROUP BY PAGE_TITLE
        ORDER BY number_of_pageviews DESC
        LIMIT 10
    """)

df.registerTempTable("pageviews_by_title")

display(spark.sql("SELECT * FROM pageviews_by_title"))

PAGE_TITLE,number_of_pageviews
Snowplow: behavioral data creation leader,868
Get started with your Snowplow BDP demo | Snowplow,420
What is behavioral data and why is it important? | Snowplow,333
Use Cases Archive | Snowplow,327
White paper: Introductory guide to data modeling | Snowplow,286
Treating data as a product,266
A behavioral data management platform designed to evolve with your business | Snowplow,247
Explore snowplow data part 1 | Snowplow,244
Life after Universal Analytics: Why choose Snowplow? | Snowplow,239
The leading open source behavioral data management platform| Snowplow,228


In [0]:
df = spark.sql(
    """ SELECT REFR_URLHOST, sum(PAGE_VIEWS_IN_SESSION) AS number_of_pageviews
        FROM derived.snowplow_web_page_views
        WHERE REFR_URLHOST IS NOT NULL
        -- filter data range, if needed, example: AND START_TSTAMP BETWEEN DATEADD(day, -7, GETDATE()) AND  DATEADD(day, -1, GETDATE())
        GROUP BY REFR_URLHOST
        ORDER BY number_of_pageviews DESC
        LIMIT 10
    """)

df.registerTempTable("referer")

display(spark.sql("SELECT * FROM referer"))

REFR_URLHOST,number_of_pageviews
snowplowanalytics.com,3803
www.google.com,1415
go.snowplowanalytics.com,311
docs.snowplowanalytics.com,115
duckduckgo.com,99
github.com,65
get.snowplowanalytics.com,32
www.google.co.uk,31
www.linkedin.com,29
search.brave.com,18


# SESSIONS

In [0]:
df = spark.sql(
    """ select date(START_TSTAMP) as Date,  round(avg(engaged_time_in_s), 0) as Avg_Engaged_Time
        from derived.snowplow_web_sessions
        -- filter data range, if needed, example: WHERE START_TSTAMP BETWEEN DATEADD(day, -7, GETDATE()) AND  DATEADD(day, -1, GETDATE())
        group BY date(START_TSTAMP)
        order BY date(START_TSTAMP)

    """)

df.registerTempTable("avg_duration_by_day")

display(spark.sql("SELECT * FROM avg_duration_by_day"))

Date,Avg_Engaged_Time
2022-08-19,33.0
2022-08-20,33.0
2022-08-21,42.0
2022-08-22,35.0


In [0]:
df = spark.sql(
    """ select round(avg(engaged_time_in_s), 0) as average_session_engaged_time_in_s
        from derived.snowplow_web_sessions
        -- filter data range, if needed, example: WHERE START_TSTAMP BETWEEN DATEADD(day, -7, GETDATE()) AND  DATEADD(day, -1, GETDATE())
    """)

df.registerTempTable("avg_duration")

display(spark.sql("SELECT * FROM avg_duration"))

average_session_engaged_time_in_s
37.0


In [0]:
df = spark.sql(
    """ WITH single_pageviews as
            (
                SELECT date(START_TSTAMP) as Date, count(1) AS single_pageviews
                FROM derived.snowplow_web_sessions
                WHERE PAGE_VIEWS = 1
                -- filter data range, if needed, example: AND START_TSTAMP BETWEEN DATEADD(day, -7, GETDATE()) AND  DATEADD(day, -1, GETDATE())
                GROUP BY  date(START_TSTAMP)
            )

        , total_pageviews as
            (
                SELECT date(START_TSTAMP) as Date,  sum(PAGE_VIEWS) AS total_pageviews
                FROM derived.snowplow_web_sessions
                -- filter data range, if needed, example: WHERE START_TSTAMP BETWEEN DATEADD(day, -7, GETDATE()) AND  DATEADD(day, -1, GETDATE())
                GROUP BY  date(START_TSTAMP)
            )

        SELECT total_pageviews.Date, sum(single_pageviews.single_pageviews) / sum(total_pageviews.total_pageviews) as BounceRate
        FROM total_pageviews
        JOIN single_pageviews
        ON single_pageviews.Date = total_pageviews.Date
        GROUP by 1
        ORDER by 1
    """)

df.registerTempTable("bounce_rate_by_day")

display(spark.sql("SELECT * FROM bounce_rate_by_day"))

Date,BounceRate
2022-08-19,0.38
2022-08-20,0.5227790432801822
2022-08-21,0.4465811965811966
2022-08-22,0.3


In [0]:
df = spark.sql(
    """ WITH single_pageviews as
    (SELECT DOMAIN_SESSIONID, count(1) AS single_pageviews
    FROM derived.snowplow_web_sessions
    WHERE PAGE_VIEWS = 1
    -- filter data range, if needed, example: AND START_TSTAMP BETWEEN DATEADD(day, -7, GETDATE()) AND  DATEADD(day, -1, GETDATE())
    GROUP BY DOMAIN_SESSIONID),

total_pageviews as
    (
        SELECT DOMAIN_SESSIONID, sum(PAGE_VIEWS)  as total_pageviews
        FROM  derived.snowplow_web_sessions
        -- filter data range, if needed, example: WHERE START_TSTAMP BETWEEN DATEADD(day, -7, GETDATE()) AND  DATEADD(day, -1, GETDATE())
        GROUP BY DOMAIN_SESSIONID
    )

select ROUND(sum(single_pageviews.single_pageviews) / sum(total_pageviews.total_pageviews), 2) AS BounceRate
from  total_pageviews
left join single_pageviews
on total_pageviews.DOMAIN_SESSIONID = single_pageviews.DOMAIN_SESSIONID

    """)

df.registerTempTable("bounce_rate")

display(spark.sql("SELECT * FROM bounce_rate"))

BounceRate
0.48


In [0]:
df = spark.sql(
    """ SELECT date(START_TSTAMP) as Date, count(1) AS number_of_sessions
        FROM derived.snowplow_web_sessions
        -- filter data range, if needed, example: WHERE START_TSTAMP BETWEEN DATEADD(day, -7, GETDATE()) AND  DATEADD(day, -1, GETDATE())
        group BY date(START_TSTAMP)
        order BY date(START_TSTAMP)

    """)

df.registerTempTable("num_sess_by_day")

display(spark.sql("SELECT * FROM num_sess_by_day"))

Date,number_of_sessions
2022-08-19,24
2022-08-20,592
2022-08-21,564
2022-08-22,21


In [0]:
df = spark.sql(
    """ SELECT count(1) AS number_of_sessions
        FROM derived.snowplow_web_sessions
        -- filter data range, if needed, example: WHERE START_TSTAMP BETWEEN DATEADD(day, -7, GETDATE()) AND  DATEADD(day, -1, GETDATE())
    """)

df.registerTempTable("num_sessions")

display(spark.sql("SELECT * FROM num_sessions"))

number_of_sessions
1201


In [0]:
df = spark.sql(
    """ select device_family, count(*) as number_of_sessions
        from derived.snowplow_web_sessions
        -- filter data range, if needed, example: WHERE START_TSTAMP BETWEEN DATEADD(day, -7, GETDATE()) AND  DATEADD(day, -1, GETDATE())
        group by 1
        order by 2 desc
        limit 8
    """)

df.registerTempTable("sess_by_device")

display(spark.sql("SELECT * FROM sess_by_device"))

device_family,number_of_sessions
Other,346
Mac,325
iPhone,218
Samsung SM-N900T,9
iPad,8
Generic Smartphone,7
Pixel 6 Pro,5
XiaoMi Redmi Note 9 Pro,5


In [0]:
import plotly.graph_objects as go
import urllib.request
import json

df = spark.sql(
    """ select GEO_COUNTRY, count(1) as NUMBER_OF_SESSIONS
        from derived.snowplow_web_sessions
        where geo_country is not null
        -- filter data range, if needed, example: AND START_TSTAMP BETWEEN DATEADD(day, -7, GETDATE()) AND  DATEADD(day, -1, GETDATE())
        group by 1
        order by 2 desc
    """)

df = df.toPandas()

def data_by_country(df):
    url = urllib.request.urlopen("http://country.io/iso3.json")
    country_codes = json.loads(url.read().decode())
    countries = df.groupby(["GEO_COUNTRY"], as_index=False).sum()
    countries["ISO_3"] = countries["GEO_COUNTRY"].map(country_codes)
    return countries


sessions_iso3 = data_by_country(df)

fig = go.Figure(
    data=go.Choropleth(
        locations=sessions_iso3["ISO_3"],
        z=sessions_iso3["NUMBER_OF_SESSIONS"],
        colorscale="Blues",
        showscale=False,             
    )
)

fig.update_geos(
    visible=False,
    resolution=50,
    showcountries=True,
    countrycolor="LightGrey",
)
fig.update_layout(showlegend=False, dragmode=False)

fig.update_layout(
    height=300,
    width=700,
    margin={"l": 20, "r": 20, "t": 25, "b": 0},
    legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99),
)

# fig.update_layout(title_text="Sessions by Country")
fig.show()

# USERS

In [0]:
df = spark.sql(
    """ select count(1) as number_of_users
        from derived.snowplow_web_users
        -- filter data range, if needed, example: WHERE START_TSTAMP BETWEEN DATEADD(day, -7, GETDATE()) AND  DATEADD(day, -1, GETDATE())
    """)

df.registerTempTable("users")

display(spark.sql("SELECT * FROM users"))

number_of_users
1127
